# Exploring the Impacts of Architecture and Scale on GNN Performance on Relational Data
By: Joseph Guman, Atindra Jha, and Christopher Pondoc

## Introduction
Welcome back to Relbench! In this tutorial, we'll dive a bit deeper into the benchmark + Relational Deep Learning and explore several choices around architecture, scale, and generalizability. In particular, we'll look to answer the following questions:

1. Can we train our Relational Deep Learning on one entity classification task and expect strong zero-shot performance on another entity classification task? What happens if we finetune the model?
2. How does our choice of using embedding models to generate expressive node features impact our performance on node classification tasks?
3. How can we alter and/or extend the architecture of our existing Relational Deep Learning model to improve performance on different tasks?

This notebook already assumes you've looked through the tutorials on [loading in data](https://github.com/snap-stanford/relbench/blob/main/tutorials/load_data.ipynb) and [training a model](https://github.com/snap-stanford/relbench/blob/main/tutorials/train_model.ipynb), as our walkthrough uses those guides as a launchpad to explore deeper questions. If you haven't had a chance to look through those notebooks, we suggest starting there first.

With all that being said, let's get started!

## Question 0: Why Relational Deep Learning
Before getting started, let's motivate the use of Relational Deep Learning by defining a non-GNN baseline to compare against. One of the main issues with older methods -- such as tabular methods or even just using standard statistical ML algorithms -- is that they either only work on one table or require lots of heavy feature engineering to reap the benefits of GNNs. On the other hand, Relational Deep Learning allows us to learn directly on relational data without feature engineering.

For our baseline, let's use a classical statiscal model. In particular, let's use [LightGBM](https://lightgbm.readthedocs.io/en/stable/), which is a gradient boosting framework that uses tree-based learning algorithms.

In [1]:
# Run LightGBM baseline
from src.models.baseline import train_model

train_metrics, val_metrics, test_metrics = train_model("rel-f1", "driver-dnf")

# Print corresponding statistics
print(f"Train: {train_metrics}")
print(f"Val: {val_metrics}")
print(f"Test: {test_metrics}")

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loading Database object from /home/cpondoc/.cache/relbench/rel-f1/db...
Done in 0.02 seconds.


Embedding raw data in mini-batch: 100%|██████████| 3/3 [00:00<00:00, 226.14it/s]
[I 2024-12-06 15:32:44,623] A new study created in memory with name: no-name-f269b952-6b5d-49f7-a100-ecb3c696f04c
/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-06 15:32:48,435] Trial 0 finished with value: 0.6722902494331067 and parameters: {'max_depth': 4, 'learning_rate': 0.021816611956758554, 'num_leaves': 811, 'subsample': 0.17281654449169184, 'colsample_bytree': 0.2237425286291762, 'lambda_l1': 0.000373151191600059, 'lambda_l2': 0.008031010523910323, 'min_data_in_leaf': 10}. Best is trial 0 with value: 0.6722902494331067.
/home/cpondoc/classes/c

Train: {'average_precision': np.float64(0.9648215170792156), 'accuracy': 0.8803785820699325, 'f1': np.float64(0.9363843967003775), 'roc_auc': np.float64(0.7986179326016077)}
Val: {'average_precision': np.float64(0.8824602257190515), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6759727891156463)}
Test: {'average_precision': np.float64(0.8438215523942226), 'accuracy': 0.7051282051282052, 'f1': np.float64(0.8270676691729323), 'roc_auc': np.float64(0.7202215390621187)}


Great! It looks like just by taking together all of raw tables and merging them together, we're able to achieve reasonable baseline performance. Let's see if we can do better without having to merge together all of the tables, though, with Relational Deep Learning!

## Question 1: Can we generalize?
Let's take a look at our first question, which involves looking at whether our Relational Deep Learning model can generalize to other tasks with/without finetuning.

Let's first start by looking setting up Relbench. As with the other tutorials, we're taking a look at the `rel-f1` dataset and focusing on node classification tasks. We'll begin by training a model on the `driver-dnf` task, which predicts whether a driver will not finish a race in the next month.

In [2]:
from src.tasks.tasks import initialize_task, db_to_graph
import torch
from torch.nn import BCEWithLogitsLoss
from torch_geometric.seed import seed_everything

# Set up dataset and task, define metrics and loss
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)
loss_fn = BCEWithLogitsLoss()

# Set up device
seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We can then preprocess all of our Relbench data.

In [3]:
import os
from relbench.modeling.graph import make_pkey_fkey_graph
from torch_frame.config.text_embedder import TextEmbedderConfig
from src.embeddings.glove import GloveTextEmbedding

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
root_dir = "./data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Next, let's load in the data and have our model set up.

In [4]:
from src.models.loader import get_loader
from src.models.rdl.graph_sage import RDLModel

# Set up data loader and model
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)

# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

Finalize, let's initialize our training run, and evaluate our model!

In [5]:
from src.models.training import eval_model, training_run

# Get model after a training run
state_dict = training_run(
    model, device, optimizer, task, loader_dict, val_table, loss_fn, entity_table
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
eval_model(model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:03<00:00,  7.35it/s]


Epoch: 01, Train loss: 0.37068689340108035, Val metrics: {'average_precision': np.float64(0.8413505292920975), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6053696145124716)}


100%|██████████| 23/23 [00:02<00:00,  8.70it/s]


Epoch: 02, Train loss: 0.34882388496971417, Val metrics: {'average_precision': np.float64(0.8726809886943294), 'accuracy': 0.7402826855123675, 'f1': np.float64(0.8470343392299687), 'roc_auc': np.float64(0.6628208616780045)}


100%|██████████| 23/23 [00:02<00:00,  8.69it/s]


...

Epoch: 08, Train loss: 0.29171781351455367, Val metrics: {'average_precision': np.float64(0.8946805587083126), 'accuracy': 0.6731448763250883, 'f1': np.float64(0.7718865598027127), 'roc_auc': np.float64(0.6937687074829932)}


100%|██████████| 23/23 [00:02<00:00,  8.62it/s]


Epoch: 09, Train loss: 0.28480575834980837, Val metrics: {'average_precision': np.float64(0.8946204837647489), 'accuracy': 0.7314487632508834, 'f1': np.float64(0.8276643990929705), 'roc_auc': np.float64(0.6991383219954648)}


100%|██████████| 23/23 [00:02<00:00,  8.58it/s]


Epoch: 10, Train loss: 0.285419514504441, Val metrics: {'average_precision': np.float64(0.8984329689565649), 'accuracy': 0.6236749116607774, 'f1': np.float64(0.7201051248357424), 'roc_auc': np.float64(0.7017142857142856)}
Best val metrics: {'average_precision': np.float64(0.8967739202997403), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.703092970521542)}
Best test metrics: {'average_precision': np.float64(0.8485722598090208), 'accuracy': 0.7051282051282052, 'f1': np.float64(0.8270676691729323), 'roc_auc': np.float64(0.7220709510564582)}


As we can see, we are able to roughly replicate the results from the [core Relbench paper](https://huggingface.co/spaces/relbench/leaderboard). However, do the results generalize? To do so, let's load in the data for the other entity classification task within `rel-f1` -- `driver-top3` -- and see how we do. 

In [6]:
# Reuse functions to set up `driver-top3 task`
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-top3"
)
db, col_to_stype_dict = db_to_graph(dataset)
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
model.load_state_dict(state_dict)
eval_model(model, loader_dict, "test", task, device, None)

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Best test metrics: {'average_precision': np.float64(0.11946463380334034), 'accuracy': 0.1763085399449036, 'f1': np.float64(0.2997658079625293), 'roc_auc': np.float64(0.21689485785953175)}


Unfortunately, trying out our model zero-shot does not yield amazing results. However, what happens if we use this model as a starting point for finetuning on the task? Let's experiment on fine-tuning this model with fewer epochs on the `driver-top3` task and checking its performance.

In [7]:
# Get model after a training run
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
state_dict = training_run(
    model,
    device,
    optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=5,
    state_dict=state_dict,
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
eval_model(model, loader_dict, "test", task, device, None)

100%|██████████| 3/3 [00:00<00:00,  7.14it/s]


Epoch: 01, Train loss: 1.627909775011115, Val metrics: {'average_precision': np.float64(0.1522469539811377), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.36931787640429303)}


100%|██████████| 3/3 [00:00<00:00,  7.42it/s]


Epoch: 02, Train loss: 0.4858022241661131, Val metrics: {'average_precision': np.float64(0.17421820768804014), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.46453208148931213)}


100%|██████████| 3/3 [00:00<00:00,  7.41it/s]


Epoch: 03, Train loss: 0.4729988264623432, Val metrics: {'average_precision': np.float64(0.20508834466628012), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.5640196377058286)}


100%|██████████| 3/3 [00:00<00:00,  7.40it/s]


Epoch: 04, Train loss: 0.47054723427195244, Val metrics: {'average_precision': np.float64(0.2944981584964439), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6166884664313487)}


100%|██████████| 3/3 [00:00<00:00,  7.39it/s]


Epoch: 05, Train loss: 0.448161677836667, Val metrics: {'average_precision': np.float64(0.2945495242915126), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6247693107093584)}
Best val metrics: {'average_precision': np.float64(0.29902345995348373), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6250918277758865)}
Best test metrics: {'average_precision': np.float64(0.2853605701210025), 'accuracy': 0.8236914600550964, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.7279211956521738)}


Nice! It looks like after we finetune even after just one epoch. we're able to practically replicate the Relbench results. Finally, let's compare this approach to simply training on the task from scratch.

In [8]:
# Define a new model, don't load in old weights.
base_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
base_optimizer = torch.optim.Adam(base_model.parameters(), lr=0.005)
base_state_dict = training_run(
    base_model,
    device,
    base_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
base_model.load_state_dict(base_state_dict)

# Evaluate on val and test set
eval_model(base_model, loader_dict, "val", task, device, val_table)
eval_model(base_model, loader_dict, "test", task, device, None)

100%|██████████| 3/3 [00:00<00:00,  7.29it/s]


Epoch: 01, Train loss: 0.5626445334881037, Val metrics: {'average_precision': np.float64(0.2271742133087044), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.5867302144738492)}


100%|██████████| 3/3 [00:00<00:00,  7.33it/s]


Epoch: 02, Train loss: 0.4550412724349381, Val metrics: {'average_precision': np.float64(0.288867727900172), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.5994517209869022)}


100%|██████████| 3/3 [00:00<00:00,  7.38it/s]


...

Epoch: 08, Train loss: 0.42793293245527714, Val metrics: {'average_precision': np.float64(0.30745570396109817), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6284872157818351)}


100%|██████████| 3/3 [00:00<00:00,  7.36it/s]


Epoch: 09, Train loss: 0.412765081423966, Val metrics: {'average_precision': np.float64(0.3225662811786547), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6512694630090842)}


100%|██████████| 3/3 [00:00<00:00,  7.33it/s]


Epoch: 10, Train loss: 0.38622126822461045, Val metrics: {'average_precision': np.float64(0.2571774250824307), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6542437870670657)}
Best val metrics: {'average_precision': np.float64(0.2571962086061911), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6543154575262942)}
Best test metrics: {'average_precision': np.float64(0.4109429240385473), 'accuracy': 0.8236914600550964, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.8192281563545151)}


Ultimately, we don't see much of a difference from starting from random weights to using a model pre-initialized from another entity classification task.

### Challenge
Does this trend necessarily work on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!

## Question 2: Different expressiveness of node features?
Next, let's take a look at using different embedding models for node features.

The embedding models are used to help turn the tabular data into usable node features. In the Relbench tutorial, the team uses GloVe embeddings, but the paper also mentions utilizing BERT-style embeddings. In traditional NLP, BERT embeddings are much more popular given that they are contextual -- the vector representation depends on the surrounding words, compared to static embeddings used by GloVe -- and can handle words outside of their vocabulary. In addition, their embedding size is $768$ compared to GloVe's $300$, which introduces an opportunity for more expressiveness.

As an investigation, let's switch out our GloVe embedding model with BERT and retrain a new model from scratch on the `driver-dnf` task.

In [9]:
from src.embeddings.bert import BertTextEmbedding

dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=BertTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)

# Initialize new, untrained model using BERT embeddings
bert_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
bert_optimizer = torch.optim.Adam(bert_model.parameters(), lr=0.005)
bert_state_dict = training_run(
    bert_model,
    device,
    bert_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
bert_model.load_state_dict(bert_state_dict)

# Evaluate on val and test set
eval_model(bert_model, loader_dict, "val", task, device, val_table)
eval_model(bert_model, loader_dict, "test", task, device, None)

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Epoch: 01, Train loss: 0.3697627618361727, Val metrics: {'average_precision': np.float64(0.8639765121043439), 'accuracy': 0.7561837455830389, 'f1': np.float64(0.8565488565488566), 'roc_auc': np.float64(0.664108843537415)}


100%|██████████| 23/23 [00:02<00:00,  8.54it/s]


Epoch: 02, Train loss: 0.32239440482574344, Val metrics: {'average_precision': np.float64(0.8884675620196203), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6762630385487529)}


100%|██████████| 23/23 [00:02<00:00,  8.52it/s]


...

Epoch: 08, Train loss: 0.2882502089535236, Val metrics: {'average_precision': np.float64(0.9103748684769377), 'accuracy': 0.6130742049469965, 'f1': np.float64(0.6928471248246845), 'roc_auc': np.float64(0.7323537414965987)}


100%|██████████| 23/23 [00:02<00:00,  8.75it/s]


Epoch: 09, Train loss: 0.2890426506634484, Val metrics: {'average_precision': np.float64(0.898380592555454), 'accuracy': 0.7226148409893993, 'f1': np.float64(0.8180764774044033), 'roc_auc': np.float64(0.7067755102040816)}


100%|██████████| 23/23 [00:02<00:00,  8.78it/s]


Epoch: 10, Train loss: 0.28385589989941185, Val metrics: {'average_precision': np.float64(0.902983368575687), 'accuracy': 0.6749116607773852, 'f1': np.float64(0.773955773955774), 'roc_auc': np.float64(0.7134875283446713)}
Best val metrics: {'average_precision': np.float64(0.910340172054631), 'accuracy': 0.6183745583038869, 'f1': np.float64(0.6974789915966386), 'roc_auc': np.float64(0.7321904761904762)}
Best test metrics: {'average_precision': np.float64(0.8346510860029261), 'accuracy': 0.6680911680911681, 'f1': np.float64(0.7268464243845252), 'roc_auc': np.float64(0.7099497389352462)}


We ultimately don't see that drastic of a difference between using BERT embeddings and GloVe embeddings. Despite being trained differently, the fact that the models are close in size and perform similarly on [general embedding benchmarks](https://huggingface.co/spaces/mteb/leaderboard) may suggest that the results will not be that drastic. 

### Challenge
We encourage you to try larger models with even larger embedding dimensions -- to do so, use our `CustomTextEmbedding` class! To use this class, import it as below, and then specify the name of a model as used on HuggingFace:

```python
from src.embeddings.custom import CustomTextEmbedding
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=CustomTextEmbedding(model_name=<INSERT_HUGGINGFACE_MODEL_HERE>, device=device), batch_size=128
)
```

## Question 3: Different RDL model architectures?
Finally, we experiment with different RDL model architectures. In particular, we investigate what happens as we add or subtract GNN layers from our model.

First, we double the number of GNN layers in our RDL pipeline, moving from `num_layers=2` to `num_layers=4`. The idea is that by adding more layers, we can create a more expressive network that can understand more complex relationships.

In [10]:
# Define a new model, don't load in old weights.
deep_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=4,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
deep_optimizer = torch.optim.Adam(deep_model.parameters(), lr=0.005)
deep_state_dict = training_run(
    deep_model,
    device,
    deep_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
deep_model.load_state_dict(deep_state_dict)

# Evaluate on val and test set
eval_model(deep_model, loader_dict, "val", task, device, val_table)
eval_model(deep_model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:03<00:00,  6.43it/s]


Epoch: 01, Train loss: 0.3881759429837744, Val metrics: {'average_precision': np.float64(0.8344568772026486), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.5962086167800453)}


100%|██████████| 23/23 [00:03<00:00,  6.45it/s]


Epoch: 02, Train loss: 0.3540312760410713, Val metrics: {'average_precision': np.float64(0.8367642571021501), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.5942675736961451)}


100%|██████████| 23/23 [00:03<00:00,  6.43it/s]


...

Epoch: 08, Train loss: 0.30175078662143207, Val metrics: {'average_precision': np.float64(0.9021201170865285), 'accuracy': 0.726148409893993, 'f1': np.float64(0.8349307774227902), 'roc_auc': np.float64(0.6952743764172336)}


100%|██████████| 23/23 [00:03<00:00,  6.45it/s]


Epoch: 09, Train loss: 0.30035592608422174, Val metrics: {'average_precision': np.float64(0.8909850809150603), 'accuracy': 0.7703180212014135, 'f1': np.float64(0.8681541582150102), 'roc_auc': np.float64(0.6942766439909297)}


100%|██████████| 23/23 [00:03<00:00,  6.41it/s]


Epoch: 10, Train loss: 0.2977251843376693, Val metrics: {'average_precision': np.float64(0.9000885935377556), 'accuracy': 0.7137809187279152, 'f1': np.float64(0.812933025404157), 'roc_auc': np.float64(0.7027482993197279)}
Best val metrics: {'average_precision': np.float64(0.9007444543861309), 'accuracy': 0.7102473498233216, 'f1': np.float64(0.8106235565819861), 'roc_auc': np.float64(0.7047619047619048)}
Best test metrics: {'average_precision': np.float64(0.8497889535900964), 'accuracy': 0.6866096866096866, 'f1': np.float64(0.7741273100616016), 'roc_auc': np.float64(0.7155272532084127)}


As we see, training using $4$ layers actually makes the model perform worse over time. Thus, given the simplicity of the task and the size of dataset, it is likely that we are overfitting.

Given that using double the amount of layers leads to less optimal results, we can try the opposite strategy and halve the number of layers in the network.

In [11]:
# Define a new model, don't load in old weights.
shallow_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=1,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
shallow_optimizer = torch.optim.Adam(shallow_model.parameters(), lr=0.005)
shallow_state_dict = training_run(
    shallow_model,
    device,
    shallow_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
shallow_model.load_state_dict(shallow_state_dict)

# Evaluate on val and test set
eval_model(shallow_model, loader_dict, "val", task, device, val_table)
eval_model(shallow_model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:02<00:00, 11.05it/s]


Epoch: 01, Train loss: 0.3751864998777913, Val metrics: {'average_precision': np.float64(0.8408527322359316), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6049342403628117)}


100%|██████████| 23/23 [00:02<00:00, 11.05it/s]


Epoch: 02, Train loss: 0.3511478803309495, Val metrics: {'average_precision': np.float64(0.8621855134101036), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6504126984126983)}


100%|██████████| 23/23 [00:02<00:00, 11.08it/s]


...

Epoch: 08, Train loss: 0.2925353284098287, Val metrics: {'average_precision': np.float64(0.9067794340316953), 'accuracy': 0.784452296819788, 'f1': np.float64(0.8737060041407867), 'roc_auc': np.float64(0.7194557823129252)}


100%|██████████| 23/23 [00:02<00:00, 10.89it/s]


Epoch: 09, Train loss: 0.2905816652717144, Val metrics: {'average_precision': np.float64(0.9059182089561669), 'accuracy': 0.7756183745583038, 'f1': np.float64(0.8733798604187437), 'roc_auc': np.float64(0.7191836734693879)}


100%|██████████| 23/23 [00:02<00:00, 11.09it/s]


Epoch: 10, Train loss: 0.28373547845243646, Val metrics: {'average_precision': np.float64(0.904110681542319), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8717948717948718), 'roc_auc': np.float64(0.7184761904761905)}
Best val metrics: {'average_precision': np.float64(0.9078009395566446), 'accuracy': 0.7579505300353356, 'f1': np.float64(0.849615806805708), 'roc_auc': np.float64(0.7244444444444444)}
Best test metrics: {'average_precision': np.float64(0.8490741640211072), 'accuracy': 0.7236467236467237, 'f1': np.float64(0.8207024029574861), 'roc_auc': np.float64(0.7179183135704875)}


Interestingly enough, we see that even with half the number of layers, we do just about the same as with double the number of layers. Once again, this might be more task-specific as opposed to a general conclusion about GNNs and the RDL pipeline.

### Challenge
Does this trend necessarily hold on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!

## Question 4: What about different graph layers?
We can also just try using different graph layers, featured in PyG. Below is GCN.

In [12]:
# Define a new model, don't load in old weights.
from src.models.rdl.gat import RDLGATModel

gcn_model = RDLGATModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.005)
gcn_state_dict = training_run(
    gcn_model,
    device,
    gcn_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
gcn_model.load_state_dict(gcn_state_dict)

# Evaluate on val and test set
eval_model(gcn_model, loader_dict, "val", task, device, val_table)
eval_model(gcn_model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:03<00:00,  7.41it/s]


Epoch: 01, Train loss: 0.37916472586069816, Val metrics: {'average_precision': np.float64(0.8277198555546621), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.5819047619047618)}


100%|██████████| 23/23 [00:03<00:00,  7.49it/s]


Epoch: 02, Train loss: 0.35965579074330256, Val metrics: {'average_precision': np.float64(0.8527877529438107), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6434739229024944)}


100%|██████████| 23/23 [00:03<00:00,  7.50it/s]


...

Epoch: 08, Train loss: 0.287606537114852, Val metrics: {'average_precision': np.float64(0.8769214089777151), 'accuracy': 0.7685512367491166, 'f1': np.float64(0.8601921024546425), 'roc_auc': np.float64(0.6870839002267572)}


100%|██████████| 23/23 [00:03<00:00,  7.54it/s]


Epoch: 09, Train loss: 0.2846338074011531, Val metrics: {'average_precision': np.float64(0.8892736560649124), 'accuracy': 0.7685512367491166, 'f1': np.float64(0.8552486187845304), 'roc_auc': np.float64(0.7104852607709752)}


100%|██████████| 23/23 [00:03<00:00,  7.51it/s]


Epoch: 10, Train loss: 0.27974842263635563, Val metrics: {'average_precision': np.float64(0.8890342528582273), 'accuracy': 0.7597173144876325, 'f1': np.float64(0.8550106609808102), 'roc_auc': np.float64(0.7074376417233559)}
Best val metrics: {'average_precision': np.float64(0.8916694835622336), 'accuracy': 0.7579505300353356, 'f1': np.float64(0.8462401795735129), 'roc_auc': np.float64(0.7143673469387753)}
Best test metrics: {'average_precision': np.float64(0.8145662802287866), 'accuracy': 0.7094017094017094, 'f1': np.float64(0.7806451612903226), 'roc_auc': np.float64(0.6876787195627776)}
